In [1]:
import pandas as pd

In [2]:
players = pd.read_csv('Data/Players.csv')
matches = pd.read_csv('Data/Matches.csv')

In [8]:
winner_counts = matches["Winner"].value_counts().reset_index()
winner_counts.columns = ["ShortName", "Wins"]
winner_counts

,ShortName,Wins
0,XXX,3
1,CG,2
2,NN,1


In [6]:
looser_counts = matches["Looser"].value_counts().reset_index()
looser_counts.columns = ["ShortName", "Looser"]
looser_counts

,ShortName,Looser
0,CG,2
1,XXX,2
2,NN,2


In [7]:
results = players
results = pd.merge(results, winner_counts, on="ShortName", how="left")
results = pd.merge(results, looser_counts, on="ShortName", how="left")
results.sort_values(by="Wins", ascending=False)

,ShortName,Name,Country,Wins,Looser
0,XXX,xxxprod,Austria,3,2
1,CG,Captain Gaming,USA,2,2
2,NN,NoName,NoCountry,1,2
